In [62]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm

In [141]:
geton_station_id = 'geton_station_id'
geton_station_name = 'geton_stataion_name'
geton_station_longitude = 'geton_station_longitude'
geton_station_latitude = 'geton_station_latitude'
user_count = 'user_count'

usage = 'usage' # not excel column, for new DataFrame object

#### 1. 입력 기간 사이에 테그 데이터 누계

In [64]:
def make_path(date):
    root_path = "C:/tb_bus_user_usage"
    base_name = "tb_bus_user_usage_"
    extender = ".csv"
    
    date = str(date)
    y = date[2:4]
    m = date[5:7]
    d = date[8:10]
    
    file_name = root_path+"/"+base_name+y+m+d+extender
    return file_name

In [95]:
def analyze_how_many_is_station_used(df):
    df = pd.DataFrame(df[geton_station_id].value_counts())
    df.index.name = geton_station_id
    df = df.rename(columns={geton_station_id:usage})
    return df

def analyze_how_many_did_user_use_station(df):
    df = df[[geton_station_id, user_count]].groupby(geton_station_id).sum()
    df.index.name = geton_station_id
    df = df.rename(columns={user_count:usage})
    return df

In [66]:
def combine_result(df_a, df_b):
    df_a_ = pd.DataFrame(index = set(df_b.index)-set(df_a.index))
    df_b_ = pd.DataFrame(index = set(df_a.index)-set(df_b.index))

    df_a_[usage] = 0
    df_b_[usage] = 0
    
    df_a = pd.concat([df_a, df_a_])
    df_b = pd.concat([df_b, df_b_])
    
    return df_a.add(df_b)

In [150]:
######################
def analyze_total_usage(start_date, end_date, scale = "station"):# scale = ["station" | "user"]
    if(scale == "station"):# select analyze function and scale column
        scale = geton_station_name
        analyze_func = analyze_how_many_is_station_used
    elif(scale == "user"):
        scale = user_count
        analyze_func = analyze_how_many_did_user_use_station

    extraction_columns = [geton_station_id, geton_station_name, geton_station_latitude, geton_station_longitude]
        
    df = pd.read_csv(make_path(start_date), encoding='utf-8')
    used_station = df[extraction_columns]
    station_usage = pd.DataFrame(analyze_func(df))

    for day in tqdm(range((end_date-start_date).days)):
        date = start_date + datetime.timedelta(days = day+1)
        df = pd.read_csv(make_path(date), encoding='utf-8')
        station_usage = combine_result(station_usage, analyze_func(df))
        used_station = used_station.append(df[extraction_columns])
        used_station = used_station.drop_duplicates(used_station.columns)
    station_usage = station_usage.reset_index().rename(columns = {"index": geton_station_id})
    result = pd.merge(used_station, station_usage, on=geton_station_id)
    print(result)
    result = result.dropna(axis = 0)
    result = result[(result[geton_station_longitude] != "nan") & (result[geton_station_latitude] != "nan")]
    result = result.sort_values(by=usage, ascending = False)
    return result

In [68]:
def make_new_path(start_date, end_date):
    s_d = str(start_date)
    e_d = str(end_date)

    s_y, s_m, s_d = s_d[2:4], s_d[5:7], s_d[8:10]
    e_y, e_m, e_d = e_d[2:4], e_d[5:7], e_d[8:10]
    
    return "tag_usage_data_"+s_y+s_m+s_d+"_"+e_y+e_m+e_d+".csv"

#### 2. 주소 컬럼추가

In [69]:
import xml.etree.ElementTree as elemTree
import requests
def get_address(loc_x, loc_y, min_x = 126.531891, min_y = 33.399409, key = "E20F6493-C13D-3F6F-AC90-D5BB2F239901"):
    loc_x = round(float(loc_x), 7)
    loc_y = round(float(loc_y), 7)
    url_form = "http://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point={},{}&format=xml&type=both&zipcode=true&simple=false&key={}"
    url = url_form.format(loc_x, loc_y, key)
    response = requests.get(url)
    tree = elemTree.fromstring(response.text)
    branch = ""
    try:
#     road = tree.find("result").find("item[2]").find("text").text
        branch = tree.find("result").find("item[1]").find("text").text
    except:
        print(loc_x, loc_y)
        x_left = str(int(loc_x))
        y_left = str(int(loc_y))
        
        
        if min_x < loc_x:
            x_right = str((int(loc_x*100000)-2)%100000)
        else:
            x_right = str((int(loc_x*100000)+2)%100000)
            
        if min_y < loc_y:
            y_right = str((int(loc_y*100000)-2)%100000)
        else:
            y_right = str((int(loc_y*100000)+2)%100000)
            
        loc_x = float(x_left+"."+x_right)
        loc_y = float(y_left+"."+y_right)
        return get_address(loc_x, loc_y)
            
    return branch

# loc_x = 126.30207
# loc_y = 33.2253
# print(get_address(loc_x, loc_y))

In [158]:
def set_address_column(df):
    df = pd.DataFrame.copy(df)
    df["STATION_ADDRESS"] = ""
    for i in tqdm(df.index):
        df.loc[i, "STATION_ADDRESS"] = get_address(float(df.loc[i, "STATION_X"]), float(df.loc[i, "STATION_Y"]))
    return df
# usage_df = set_address_column(usage_df)
# usage_df

#### main

In [159]:
### setting clasification period(start, end)
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)


### accumulating
usage_df = analyze_total_usage(start_date, end_date, scale = "user")


### calculate mean of usage
# period = int((end_date-start_date).days)+1
# usage_df[usage] = usage_df[usage].apply(lambda x : int(x/period))


### new naming
usage_df = usage_df.rename(columns = {geton_station_id :'STATION_ID',
                          geton_station_name: "STATION_NAME", 
                          geton_station_latitude:"STATION_Y",
                          geton_station_longitude:"STATION_X",
                          usage:"STATION_USAGE"})


### setting address column
usage_df = set_address_column(usage_df)


### sorting
usage_df = usage_df.sort_values(by="STATION_ID")


### stroing
usage_df.to_csv(make_new_path(start_date, end_date), encoding="CP949", index=False)

  2%|█▊                                                                                 | 2/89 [00:00<00:38,  2.25it/s]D:\Users\Embedded\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [00:39<00:00,  2.24it/s]
D:\Users\Embedded\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
  0%|                                                                                 | 1/3622 [00:00<08:03,  7.48it/s]

      geton_station_id geton_stataion_name  geton_station_latitude  \
0                  988              납읍리사무소                33.43944   
1                 1130        한림환승정류장(한림리)                33.41453   
2                   47                 학원동                33.47493   
3                   20                연대마을                33.49282   
4                  174                명주주택                33.49579   
...                ...                 ...                     ...   
3621              4250               추사유배지                33.24910   
3622              3314                 산수동                33.30309   
3623              4206            크라운골프연습장                33.33965   
3624              2148             세화3리웃동네                33.32318   
3625              3559              두문포노린당                33.49630   

      geton_station_longitude  usage  
0                   126.32944   3234  
1                   126.26427  38821  
2                   126.39766   2135  
3  

  8%|██████▎                                                                        | 289/3622 [00:21<04:31, 12.27it/s]

126.61324 33.2642


 25%|████████████████████                                                           | 917/3622 [01:07<03:07, 14.46it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████▊| 3614/3622 [04:40<00:00, 13.57it/s]

126.61747 33.26697
126.61745 33.26699
126.61743 33.26701
126.61741 33.26703
126.61739 33.26705
126.61737 33.26706
126.61735 33.26708
126.61733 33.2671
126.61731 33.26712
126.61729 33.26714
126.61727 33.26716
126.61725 33.26717
126.61723 33.26719
126.61721 33.26721
126.61719 33.26723


100%|█████████████████████████████████████████████████████████████████████████████▊| 3616/3622 [04:42<00:01,  4.04it/s]

126.61717 33.26725


100%|██████████████████████████████████████████████████████████████████████████████| 3622/3622 [04:42<00:00, 12.82it/s]
